# `MySQL Dealing with Dupicate Data: with Python`

# <font color=red>Mr Fugu Data Science</font>

# (◕‿◕✿)



# If you don't have user credentials:

`heres how you can set it up`

First, don't forget to establish a connection to MySQL, I used the `mysl.server.start` on Mac since I don't have it on all the time.

[video covering credentials MuSQL and Psql](https://youtu.be/kIS58p9m9Io) | [Mysql connection and credentials setup](https://youtu.be/9kvHZnIz_zw)

In [41]:
import mysql.connector as mysql # Establish connection to mysql 

from mysql.connector import MySQLConnection # Using Class for function call

import pandas as pd  # Create Dataframe

import os,datetime

In [3]:
def os_any_dir_search(file):
    u=[]
    for p,n,f in os.walk(os.getcwd()):
        
        for a in f:
            a = str(a)
            if a.endswith(file): # can be (.csv) or a file like I did and search 
#                 print(a)
#                 print(p)
#                 t=pd.read_json(p+'/'+file)
                u.append(p+'/')
    return u

In [4]:
t=os_any_dir_search('python_to_myql_config.py')
t
os.chdir(t[0])


from python_to_myql_config import read_db_config # calling config files

db_config = read_db_config() # user credentials

conn = MySQLConnection(**read_db_config()) # establish connection

mycursor = conn.cursor() # cursor, allows us to do queries,send/retrieve data

In [5]:
os.getcwd() # get current working directory since I changed it

'/Users/zatoichi59/Desktop/youtube_vid_code'

# `Transactions Data:`

+ we want to find the duplicates and remove them or ignore them.

In [8]:
# get working directory 
w=os.path.normpath(os.getcwd() + os.sep + os.pardir)


transactions=pd.read_csv(w+'/Transaction.txt',sep='|') 
transactions.head(6)

,transactionId,custId,transactionDate,productSold,unitsSold
0,20110111-0023586,23586,2011-01-11,DETA800,176
1,20110111-0023586,23586,2011-01-11,SUPA101,193
2,20110111-0023586,23586,2011-01-11,SUPA102,157
3,20110113-0023524,23524,2011-01-13,PURA250,122
4,20110113-0023844,23844,2011-01-13,SUPA101,217
5,20110114-0023968,23968,2011-01-14,PURA100,284


In [24]:
# Find Duplicates

print(len(transactions.loc[transactions.duplicated(),:])/len(transactions)*100)

transactions.loc[transactions.duplicated(),:].head()


0.321976787719955


,transactionId,custId,transactionDate,productSold,unitsSold
9,20110119-0023582,23582,2011-01-19,SUPA101,173
10,20110119-0023582,23582,2011-01-19,SUPA101,173
37,20110206-0023586,23586,2011-02-06,SUPA101,205
130,20110315-0023429,23429,2011-03-15,DETA800,131
131,20110315-0023429,23429,2011-03-15,DETA800,131


In [38]:
# Remove Duplicates:

transactions.drop_duplicates().head()

,transactionId,custId,transactionDate,productSold,unitsSold
0,20110111-0023586,23586,2011-01-11,DETA800,176
1,20110111-0023586,23586,2011-01-11,SUPA101,193
2,20110111-0023586,23586,2011-01-11,SUPA102,157
3,20110113-0023524,23524,2011-01-13,PURA250,122
4,20110113-0023844,23844,2011-01-13,SUPA101,217


In [72]:
# Change Date string to date/time
transactions['transactionDate']=pd.to_datetime(transactions['transactionDate'],
                    infer_datetime_format=True).astype('str')
 
# transactions['transactionDate'].dtype # verifying datatype is now date


In [46]:
# Creating table as per requirement


sql ="""
CREATE TABLE Sales_Transactions(transactionId VARCHAR(255) NOT NULL, 
custId VARCHAR(255) NOT NULL,
transactionDate DATE,productSold VARCHAR(20) NOT NULL, unitsSold int)
"""

mycursor.execute(sql)


In [74]:
# Show tables in current database:

mycursor.execute("SHOW TABLES")
mycursor.fetchall()

[('cpu_purchase',),
 ('Sales_Transactions',),
 ('user_info',),
 ('user_o',),
 ('users_inf',),
 ('users_info',)]

In [ ]:
mycursor.executemany("""
INSERT INTO Sales_Transactions(transactionId, custId, 
transactionDate, productSold,unitsSold) VALUES(%s,%s,%s,%s,%s)""",
                     list(transactions.values))

# We have a few option for finding duplicates using MySQL:

+ Do we want to look by row for 1 column
+ Are there multiple columns that we need to check for matching rows?

This will depend on what you need to achieve and how the data are setup.


For our data let's think about something:

+ We may have `transactions ID's` but, *you can have multiple items sold per transaction*, if that is a real scenario for these data then you can use that an indicator of using more than 1 column for comparisons. (unless these made up data don't act like real data) 

In [48]:
transactions[:3]

,transactionId,custId,transactionDate,productSold,unitsSold
0,20110111-0023586,23586,2011-01-11,DETA800,176
1,20110111-0023586,23586,2011-01-11,SUPA101,193
2,20110111-0023586,23586,2011-01-11,SUPA102,157


Notice above that we have the same `transaction ID` for the 3 rows and same `customer ID` with 3 different items sold. 

+ This allows us to see that we can think about either column as a potential way to find duplicates using MySQL

In [84]:
# mycursor.execute("select transactionId,\
# count(transactionId),custId,count(custId) from Sales_Transactions groupby\
# transactionId,custId having (count(tranactionId)>1) and (count(custId)>1)")
# mycursor.fetchall()

In [95]:

# cpu_purchase

# [('cpu_purchase',),
#  ('Sales_Transactions',),
#  ('user_info',),
#  ('user_o',),
#  ('users_inf',),
#  ('users_info',)]


# mycursor.execute('select * from Sales_Transactions')
# mycursor.fetchall()

print(len(transactions.loc[transactions.duplicated(),:]))
# /len(transactions)*100)
len(transactions)

43


13355

In [93]:
mycursor.execute("select transactionId,count(transactionId),custId,\
count(custId), from Sales_Transactions group by transactionId,\
custId having (count(transactionId)>1) and (count(custId)>1)")
len(mycursor.fetchall())

4474

In [96]:
mycursor.execute('select * from Sales_Transactions')
len(mycursor.fetchall())

13355

# `Returns all data with numer of times duplicated`

+ This means that you have 1 entry and frequency of occurances. It keeps every occurance.

+ Not same as dataframe.duplicated() because that will either keep first or last occurance by your choice depending what you choose. So, if you have 4 occurances you will show first three or last three and df will keep one.

In [148]:
mycursor.execute("select transactionId,custId,transactionDate,productSold,unitsSold,\
count(transactionId),count(custId),count(transactionDate), \
count(productSold),count(unitsSold) from \
Sales_Transactions group by transactionId,custId, \
transactionDate,productSold,unitsSold having \
count(transactionId)>1 and count(custId)>1 and count(transactionDate)>1 and \
count(productSold)>1 and count(unitsSold)>1")
xc=mycursor.fetchall()
pd.DataFrame(xc).head()

,0,1,2,3,4,5,6,7,8,9
0,20110119-0023582,23582,2011-01-19,SUPA101,173,3,3,3,3,3
1,20110206-0023586,23586,2011-02-06,SUPA101,205,2,2,2,2,2
2,20110315-0023429,23429,2011-03-15,DETA800,131,3,3,3,3,3
3,20110319-0023370,23370,2011-03-19,SUPA104,67,3,3,3,3,3
4,20110322-0024042,24042,2011-03-22,PURA250,39,3,3,3,3,3


In [153]:
# returns all rows with duplicates including the first occurance.
print('Number of entries duplicated w/ first occurence: ',sum(pd.DataFrame(xc)[6]))
print('returns number of rows: ',len(pd.DataFrame(xc)))



Number of entries duplicated w/ first occurence:  73
returns number of rows:  30


In [167]:
# count number rows duplicated including first occurence - df rows
# returns 43 which is same as df.duplicated giving you df with first occurence subtracted

sum(pd.DataFrame(xc)[6])-len(pd.DataFrame(xc)[6])

43

In [168]:
# Verify what is going on again a little differently

tryop=transactions[transactions.duplicated(subset=['custId','transactionId',
                        'transactionDate','unitsSold','productSold'])]

print('number of dups NOT including first occurence:',
sum(tryop.groupby(['transactionId','custId']).count()['unitsSold']))

print('rows of entries, same as if you counted first occurence from DF:',
len(tryop.groupby(['transactionId','custId']).count()['unitsSold']))

number of dups NOT including first occurence: 43
rows of entries, same as if you counted first occurence from DF: 30


# `Returns Data w/o duplicates as query`

+ keeps first occurence

In [124]:
mycursor.execute("select transactionId,custId,transactionDate,productSold,unitsSold,\
count(transactionId),count(custId),count(transactionDate), \
count(productSold),count(unitsSold) from \
Sales_Transactions group by transactionId,custId, \
transactionDate,productSold,unitsSold ")
len(mycursor.fetchall())

13312

In [127]:
# keeps first occurence:

len(transactions.drop_duplicates())

13312

In [126]:
# original data length:
len(transactions)

13355

In [143]:
len(transactions[transactions.duplicated(subset=['custId','transactionId',
                        'transactionDate','unitsSold','productSold'])])

43

In [166]:
tryop=transactions[transactions.duplicated(subset=['custId','transactionId',
                        'transactionDate','unitsSold','productSold'])]

print('number of dups NOT including first occurence:',
sum(tryop.groupby(['transactionId','custId']).count()['unitsSold']))

print('rows of entries, same as if you counted first occurence from DF:',
len(tryop.groupby(['transactionId','custId']).count()['unitsSold']))

number of dups NOT including first occurence: 43
rows of entries, same as if you counted first occurence from DF: 30


# <font color=red>Like</font>, Share &

# <font color=red>SUB</font>scribe

# `Citations & Help:`

# ◔̯◔

https://www.mysqltutorial.org/mysql-delete-duplicate-rows/

https://www.tutorialspoint.com/How-to-insert-date-object-in-MySQL-using-Python

https://phoenixnap.com/kb/mysql-remove-duplicate-rows